# Import the necessary files

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

## Load MovieLens dataset

In [ ]:
DATASET_LINK='http://files.grouplens.org/datasets/movielens/ml-100k.zip'

In [ ]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -n ml-100k.zip

--2022-11-24 12:16:31--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  11.3MB/s    in 0.4s    

2022-11-24 12:16:32 (11.3 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

Loading u.info     -- The number of users, items, and ratings in the u data set.

In [ ]:
overall_stats = pd.read_csv('ml-100k/u.info', header=None)
print("Details of users, items and ratings involved in the loaded movielens dataset: ",list(overall_stats[0]))

Details of users, items and ratings involved in the loaded movielens dataset:  ['943 users', '1682 items', '100000 ratings']


In [ ]:
##item id column is renamed as movie id
column_names1 = ['user id','movie id','rating','timestamp']
dataset = pd.read_csv('ml-100k/u.data', sep='\t',header=None,names=column_names1)
dataset.head() 


,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
d = 'movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
column_names2 = d.split(' | ')
column_names2

['movie id',
 'movie title',
 'release date',
 'video release date',
 'IMDb URL',
 'unknown',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [ ]:
items_dataset = pd.read_csv('ml-100k/u.item', sep='|',header=None,names=column_names2,encoding='latin-1')
items_dataset

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
movie_dataset = items_dataset[['movie id','movie title']]
movie_dataset.head()

,movie id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


## Merging required datasets

In [ ]:
merged_dataset = pd.merge(dataset, movie_dataset, how='inner', on='movie id')
merged_dataset.head()

,user id,movie id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [ ]:
refined_dataset = merged_dataset.groupby(['user id','movie id', 'movie title'], as_index=False).agg({"rating":"mean"})

refined_dataset

,user id,movie id,movie title,rating
0,1,1,Toy Story (1995),5.0
1,1,2,GoldenEye (1995),3.0
2,1,3,Four Rooms (1995),4.0
3,1,4,Get Shorty (1995),3.0
4,1,5,Copycat (1995),3.0
...,...,...,...,...
99995,943,1067,Bottle Rocket (1996),2.0
99996,943,1074,Reality Bites (1994),4.0
99997,943,1188,Young Guns II (1990),3.0
99998,943,1228,Under Siege 2: Dark Territory (1995),3.0


## Normalization of Ratings

#### Bayesian Average

Bayesian Average is defined as:

$r_{i} = \frac{C \times m + \Sigma{\text{reviews}}}{C+N}$

where $C$ represents our confidence, $m$ represents our prior, and $N$ is the total number of reviews for movie $i$. In this case, our prior will be the average rating across all movies. By defintion, C represents "the typical dataset size". Let's make $C$ be the average number of ratings for a given movie.

In [ ]:
movie_stats = refined_dataset.groupby(['movie title', 'movie id'], as_index=False)[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()


In [ ]:
movie_stats

,,count,mean
movie title,movie id,,
'Til There Was You (1997),1300,9,2.333333
1-900 (1994),1353,5,2.600000
101 Dalmatians (1996),225,109,2.908257
12 Angry Men (1957),178,125,4.344000
187 (1997),330,41,3.024390
...,...,...,...
Young Guns II (1990),1188,44,2.772727
"Young Poisoner's Handbook, The (1995)",547,41,3.341463
Zeus and Roxanne (1997),1164,6,2.166667


In [ ]:
C = movie_stats['count'].mean()
m = movie_stats['mean'].mean()

def bayesian_avg(ratings):
    bayesian_avg = (C*m+ratings.sum())/(C+ratings.count())
    return bayesian_avg

bayesian_avg_ratings = refined_dataset.groupby([ 'movie id'])['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movie id', 'bayesian_avg']
movie_stats = movie_stats.merge(bayesian_avg_ratings, on = 'movie id')

In [ ]:
movie_stats = movie_stats.merge(movie_dataset[['movie id', 'movie title']])
movie_stats.sort_values('bayesian_avg', ascending=False).head()

,movie id,count,mean,bayesian_avg,movie title
1413,50,583,4.358491,4.239812,Star Wars (1977)
1295,318,298,4.466443,4.235186,Schindler's List (1993)
1331,64,283,4.445230,4.207526,"Shawshank Redemption, The (1994)"
275,483,243,4.456790,4.185378,Casablanca (1942)
1590,12,267,4.385768,4.147243,"Usual Suspects, The (1995)"


In [ ]:
movie_stats.sort_values('bayesian_avg', ascending=True).head()

,movie id,count,mean,bayesian_avg,movie title
866,688,44,1.840909,2.550724,Leave It to Beaver (1997)
970,687,69,2.188406,2.599239,McHale's Navy (1997)
257,122,106,2.339623,2.604245,"Cable Guy, The (1996)"
1013,890,43,1.953488,2.604903,Mortal Kombat: Annihilation (1997)
139,988,86,2.313953,2.625454,"Beautician and the Beast, The (1997)"


In [ ]:
refined_dataset = pd.merge(movie_stats, refined_dataset, how='inner', on='movie id')
refined_dataset

,movie id,count,mean,bayesian_avg,movie title_x,user id,movie title_y,rating
0,1300,9,2.333333,2.978395,'Til There Was You (1997),152,'Til There Was You (1997),4.0
1,1300,9,2.333333,2.978395,'Til There Was You (1997),178,'Til There Was You (1997),3.0
2,1300,9,2.333333,2.978395,'Til There Was You (1997),223,'Til There Was You (1997),1.0
3,1300,9,2.333333,2.978395,'Til There Was You (1997),299,'Til There Was You (1997),2.0
4,1300,9,2.333333,2.978395,'Til There Was You (1997),342,'Til There Was You (1997),1.0
...,...,...,...,...,...,...,...,...
99995,267,9,3.444444,3.124481,unknown,319,unknown,4.0
99996,267,9,3.444444,3.124481,unknown,422,unknown,4.0
99997,267,9,3.444444,3.124481,unknown,532,unknown,3.0
99998,267,9,3.444444,3.124481,unknown,833,unknown,1.0


In [ ]:
refined_dataset.drop(columns = ['count', 'mean', 'movie title_y'])

,movie id,bayesian_avg,movie title_x,user id,rating
0,1300,2.978395,'Til There Was You (1997),152,4.0
1,1300,2.978395,'Til There Was You (1997),178,3.0
2,1300,2.978395,'Til There Was You (1997),223,1.0
3,1300,2.978395,'Til There Was You (1997),299,2.0
4,1300,2.978395,'Til There Was You (1997),342,1.0
...,...,...,...,...,...
99995,267,3.124481,unknown,319,4.0
99996,267,3.124481,unknown,422,4.0
99997,267,3.124481,unknown,532,3.0
99998,267,3.124481,unknown,833,1.0


In [ ]:
refined_dataset.rename(columns = {'movie title_x':'movie title'}, inplace = True)


# Nearest Neighbour model to build user-user based collaborative Recommender System
This part of the code performs movie recommendation using KNN with input as User ID, size of the neighbourhood, and the number of movies to be recommended:


## Transformation - Reshaping the Dataframe
We need to transform the data so that each row of the dataframe represents a user and each column represents a different movie. So we want the data to be [users, movies] array if the user is the subject: ie, similar users must be found. To reshape, we pivot the dataframe to a format with users as rows and movies as columns. Not all users watch all the movies, so we can expect a lot of missing values. 

However, in the MovieLens 100k dataset, every user has rated at least 20 movies. The missing observations are filled with 0s to avoid NaN values while performing linear algebra operations (in this case, calculating cosine distances between vectors). Finally, we transform the values of the dataframe into a scipy sparse matrix for efficient calculations. This dataframe is then fed into a KNN model. 

In [ ]:
# pivot and create movie-user matrix
user_to_movie_df = refined_dataset.pivot_table(
    index='user id',
     columns='movie title',
      values='bayesian_avg').fillna(0)

user_to_movie_df.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.967475,3.935312,0.000000,0.0,3.308255,3.802382,0.000000,0.0,...,0.0,0.0,0.0,3.745881,3.159056,0.0,0.0,0.0,3.124481,0.0
2,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.914994,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.000000,0.000000,3.054962,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
5,0.0,0.0,2.967475,0.000000,0.000000,0.0,0.000000,3.802382,0.000000,0.0,...,0.0,0.0,0.0,3.745881,0.000000,0.0,0.0,0.0,3.124481,0.0


The model is reshaped so that each user has an n-dimensional rating space where n is number of movies in the dataset.

The KNN model is trained to find the most similar users to the active user given as input. The system recommends the top movies based on the rating of the similar users in the considered neighbourhood. 

In [ ]:
# transform matrix to scipy sparse matrix
user_to_movie_sparse_df = csr_matrix(user_to_movie_df.values)
user_to_movie_sparse_df

<943x1664 sparse matrix of type '<class 'numpy.float64'>'
	with 99693 stored elements in Compressed Sparse Row format>

Fitting K-Nearest Neighbours model to the scipy sparse matrix:

In [ ]:
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_to_movie_sparse_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
## function to find top n similar users of the given input user 
def get_similar_users(user, n = 5):
  ## input to this function is the user and number of top similar users you want.

  knn_input = np.asarray([user_to_movie_df.values[user-1]])  #.reshape(1,-1)
  # knn_input = user_to_movie_df.iloc[0,:].values.reshape(1,-1)
  distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
  
  print("Top",n,"users who are very much similar to the User-",user, "are: ")
  print(" ")
  for i in range(1,len(distances[0])):
    print(i,". User:", indices[0][i]+1, "separated by distance of",distances[0][i])
  return indices.flatten()[1:] + 1, distances.flatten()[1:]


Specify the user ID and number of similar users / size of neighbourhood:

In [ ]:
from pprint import pprint
user_id = 5
#print(" Few of movies seen by the User:")
#pprint(list(refined_dataset[refined_dataset['user id'] == user_id]['movie title'])[:10])
similar_user_list, distance_list = get_similar_users(user_id,5)

Top 5 users who are very much similar to the User- 778 are: 
 
1 . User: 124 separated by distance of 0.5511898942217217
2 . User: 56 separated by distance of 0.5766415784678851
3 . User: 738 separated by distance of 0.5830788407083427
4 . User: 933 separated by distance of 0.5928931458987586
5 . User: 643 separated by distance of 0.595102794856804


The neighbourhood of simialr users is now built using the KNN model. The next step is to select the top-N movies to recommend.

This can be implemented in multiple ways: by taking the average of the existing ratings given by the similar users and picking the top 10 or 15 movies to recommend to our current user.

However, a better recommendation system would weight each user's ratings based on the how close they are to the input/active user. Defining these weights would give us the accurate recommendations by eliminating the chance of decision manipulation by the users who are very far from the input user.

In [ ]:
similar_user_list, distance_list

(array([124,  56, 738, 933, 643]),
 array([0.55118989, 0.57664158, 0.58307884, 0.59289315, 0.59510279]))

In [ ]:
weightage_list = distance_list/np.sum(distance_list)
weightage_list

array([0.19013719, 0.19891695, 0.20113753, 0.20452305, 0.20528528])

Getting ratings of all movies by derived similar users

In [ ]:
mov_rtngs_sim_users = user_to_movie_df.values[similar_user_list]
mov_rtngs_sim_users

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.96747507, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 2.96747507, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
movies_list = user_to_movie_df.columns
movies_list

Index([''Til There Was You (1997)', '1-900 (1994)', '101 Dalmatians (1996)',
       '12 Angry Men (1957)', '187 (1997)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '3 Ninjas: High Noon At Mega Mountain (1998)', '39 Steps, The (1935)',
       ...
       'Yankee Zulu (1994)', 'Year of the Horse (1997)', 'You So Crazy (1994)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Poisoner's Handbook, The (1995)',
       'Zeus and Roxanne (1997)', 'unknown',
       'Á köldum klaka (Cold Fever) (1994)'],
      dtype='object', name='movie title', length=1664)

In [ ]:
print("Weightage list shape:", len(weightage_list))
print("mov_rtngs_sim_users shape:", mov_rtngs_sim_users.shape)
print("Number of movies:", len(movies_list))

Weightage list shape: 5
mov_rtngs_sim_users shape: (5, 1664)
Number of movies: 1664


**Broadcasting weightage matrix to similar user rating matrix, so that it gets compatible for matrix operations**

In [ ]:
weightage_list = weightage_list[:,np.newaxis] + np.zeros(len(movies_list))
weightage_list.shape

(5, 1664)

In [ ]:
weightage_list[:,np.newaxis]

array([[[0.19013719, 0.19013719, 0.19013719, ..., 0.19013719,
         0.19013719, 0.19013719]],

       [[0.19891695, 0.19891695, 0.19891695, ..., 0.19891695,
         0.19891695, 0.19891695]],

       [[0.20113753, 0.20113753, 0.20113753, ..., 0.20113753,
         0.20113753, 0.20113753]],

       [[0.20452305, 0.20452305, 0.20452305, ..., 0.20452305,
         0.20452305, 0.20452305]],

       [[0.20528528, 0.20528528, 0.20528528, ..., 0.20528528,
         0.20528528, 0.20528528]]])

In [ ]:
np.zeros(len(movies_list))

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
weightage_list


array([[0.19013719, 0.19013719, 0.19013719, ..., 0.19013719, 0.19013719,
        0.19013719],
       [0.19891695, 0.19891695, 0.19891695, ..., 0.19891695, 0.19891695,
        0.19891695],
       [0.20113753, 0.20113753, 0.20113753, ..., 0.20113753, 0.20113753,
        0.20113753],
       [0.20452305, 0.20452305, 0.20452305, ..., 0.20452305, 0.20452305,
        0.20452305],
       [0.20528528, 0.20528528, 0.20528528, ..., 0.20528528, 0.20528528,
        0.20528528]])

In [ ]:
new_rating_matrix = weightage_list*mov_rtngs_sim_users
mean_rating_list = new_rating_matrix.sum(axis =0)
mean_rating_list

array([0.        , 0.        , 1.19719813, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
from pprint import pprint
def recommend_movies(n):
  n = min(len(mean_rating_list),n)
  # print(np.argsort(mean_rating_list)[::-1][:n])
  pprint(list(movies_list[np.argsort(mean_rating_list)[::-1][:n]]))


In [ ]:
print("Movies recommended based on similar users are: ")
recommend_movies(10)

Movies recommended based on similar users are: 
['Star Wars (1977)',
 'Fargo (1996)',
 'Pulp Fiction (1994)',
 'Return of the Jedi (1983)',
 'Monty Python and the Holy Grail (1974)',
 'Terminator, The (1984)',
 "Schindler's List (1993)",
 'Empire Strikes Back, The (1980)',
 'Princess Bride, The (1987)',
 'African Queen, The (1951)']


# Post-processing

By overcoming drawbacks of this system, it can be made more efficient. Here are the post processing steps to make the system better:

1. Remove movies that have already been watched by the user.

2. Do not recommend movies that are not at all seen by any of the similar users.

In [ ]:
#function to remove movies already watched by the user and movies not watched by any of the similar users.
def filtered_movie_recommendations(n):
  
  first_zero_index = np.where(mean_rating_list == 0)[0][-1]
  sortd_index = np.argsort(mean_rating_list)[::-1]
  sortd_index = sortd_index[:list(sortd_index).index(first_zero_index)]
  n = min(len(sortd_index),n)
  movies_watched = list(refined_dataset[refined_dataset['user id'] == user_id]['movie title'])
  filtered_movie_list = list(movies_list[sortd_index])
  count = 0
  final_movie_list = []
  for i in filtered_movie_list:
    if i not in movies_watched:
      count+=1
      final_movie_list.append(i)
    if count == n:
      break
  if count == 0:
    print("There are no movies left which are not seen by the input users and seen by similar users. May be increasing the number of similar users who are to be considered may give a chance of suggesting an unseen good movie.")
  else:
    pprint(final_movie_list)


In [ ]:
filtered_movie_recommendations(10)

['Star Wars (1977)',
 'Fargo (1996)',
 'Return of the Jedi (1983)',
 "Schindler's List (1993)",
 'Empire Strikes Back, The (1980)',
 'Princess Bride, The (1987)',
 'African Queen, The (1951)',
 'Toy Story (1995)',
 'Contact (1997)',
 'Men in Black (1997)']


# User-User Collaborative Filtering Recommender System

A single function that takes in a user ID and size of the neighbourhood to be considered, and returns top 10 recommended movies for the user.

In [ ]:
from pprint import pprint

def recommender_system(user_id, n_similar_users, n_movies): #, user_to_movie_df, knn_model):
  
  print("Movie seen by the User:")
  pprint(list(refined_dataset[refined_dataset['user id'] == user_id]['movie title']))
  print("")

  # def get_similar_users(user, user_to_movie_df, knn_model, n = 5):
  def get_similar_users(user, n = 10):
    
    knn_input = np.asarray([user_to_movie_df.values[user-1]])
    
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
    
    print("Top",n,"users who are very much similar to the User-",user, "are: ")
    print(" ")

    for i in range(1,len(distances[0])):
      print(i,". User:", indices[0][i]+1, "separated by distance of",distances[0][i])
    print("")
    return indices.flatten()[1:] + 1, distances.flatten()[1:]


  def filtered_movie_recommendations(n = 10):
  
    first_zero_index = np.where(mean_rating_list == 0)[0][-1]
    sortd_index = np.argsort(mean_rating_list)[::-1]
    sortd_index = sortd_index[:list(sortd_index).index(first_zero_index)]
    n = min(len(sortd_index),n)
    movies_watched = list(refined_dataset[refined_dataset['user id'] == user_id]['movie title'])
    filtered_movie_list = list(movies_list[sortd_index])
    count = 0
    final_movie_list = []
    for i in filtered_movie_list:
      if i not in movies_watched:
        count+=1
        final_movie_list.append(i)
      if count == n:
        break
    if count == 0:
      print("There are no movies left which are not seen by the input users and seen by similar users. May be increasing the number of similar users who are to be considered may give a chance of suggesting an unseen good movie.")
    else:
      pprint(final_movie_list)

  similar_user_list, distance_list = get_similar_users(user_id,n_similar_users)
  weightage_list = distance_list/np.sum(distance_list)
  mov_rtngs_sim_users = user_to_movie_df.values[similar_user_list]
  movies_list = user_to_movie_df.columns
  weightage_list = weightage_list[:,np.newaxis] + np.zeros(len(movies_list))
  new_rating_matrix = weightage_list*mov_rtngs_sim_users
  mean_rating_list = new_rating_matrix.sum(axis =0)
  print("")
  print("Movies recommended based on similar users are: ")
  print("")
  filtered_movie_recommendations(n_movies)

## Demonstration of the function's results:

In [ ]:
print("Enter user id")
user_id= int(input())
print("number of similar users to be considered")
sim_users = int(input())
print("Enter number of movies to be recommended:")
n_movies = int(input())
recommender_system(user_id,sim_users,n_movies)

Enter user id
5
number of similar users to be considered
10
Enter number of movies to be recommended:
10
Movie seen by the User:
['101 Dalmatians (1996)',
 '2001: A Space Odyssey (1968)',
 'Ace Ventura: When Nature Calls (1995)',
 'Addams Family Values (1993)',
 'Adventures of Priscilla, Queen of the Desert, The (1994)',
 'Age of Innocence, The (1993)',
 'Aladdin (1992)',
 'Aladdin and the King of Thieves (1996)',
 'Alice in Wonderland (1951)',
 'Alien (1979)',
 'Aliens (1986)',
 'American Werewolf in London, An (1981)',
 "Amityville 1992: It's About Time (1992)",
 'Amityville 3-D (1983)',
 'Amityville Curse, The (1990)',
 'Amityville Horror, The (1979)',
 'Amityville II: The Possession (1982)',
 'Amityville: A New Generation (1993)',
 'Apple Dumpling Gang, The (1975)',
 'Aristocats, The (1970)',
 'Back to the Future (1985)',
 'Bastard Out of Carolina (1996)',
 'Batman (1989)',
 'Batman Forever (1995)',
 'Batman Returns (1992)',
 'Beverly Hills Cop III (1994)',
 'Beverly Hills Ninja (1